<h1 style="color:black" align="center">Решающие деревья</h1>

<h1 style="color:#008B8B">1. Определение решающего дерева</h1>

Рассмотрим бинарное дерево, в котором:

* каждой внутренней вершине $v$ приписана функция (или предикат) $\beta_v: \mathbb{X} \to \{0, 1\}$;

* каждой листовой вершине $v$ приписан прогноз $c_v \in Y$ (в случае с классификацией листу также может быть приписан вектор вероятностей).


Рассмотрим теперь алгоритм $a(x)$, который стартует из корневой вершины $v_0$ и вычисляет значение функции $\beta_{v_0}$. Если оно равно нулю, то алгоритм переходит в левую дочернюю вершину, иначе в правую, вычисляет значение предиката в новой вершине и делает переход или влево, или вправо. Процесс продолжается, пока не будет достигнута листовая вершина; алгоритм возвращает тот класс, который приписан этой вершине. Такой алгоритм называется - *бинарным решающим деревом*.

На практике в большинстве случаев используются одномерные предикаты $\beta_v$, которые сравнивают значение одного из признаков с порогом:

$\beta_v(x; j, t) = [x_j < t].$

Существуют и многомерные предикаты, например:

* линейные $\beta_v(x) = [\langle w, x \rangle < t]$;

* метрические $\beta_v(x) = [\rho(x, x_v) < t]$, где точка $x_v$ является одним из объектов выборки любой точкой признакового пространства. $\rho(x, x_v)$ - расстояние от объекта $x$ до объекта $x_v$.

Многомерные предикаты позволяют строить ещё более сложные разделяющие поверхности, но очень редко используются на практике - например, из-за того, что усиливают и без того выдающиеся способности деревьев к переобучению. Далее мы будем говорить только об одномерных предикатах.

<h1 style="color:#008B8B">2. Построение деревьев</h1>

Допутсим, у нас имеется два признака. Визуализируем точки на графике, по оси $x$ обозначим признак $x_1$, по оси $y$ обозначим признак $x_2$. Тогда при построении бинарного дерева, мы начнем разделять плоскость полосами на небольшие части. Можно показать, что если нет двух объектов с одинаковыми признаками, но разными классами, тогда можно получить нулевую ошибку на обучающей выборке. Линейные модели являются куда более простыми, ибо они не способы переобучиться настолько сильно.

Пример переобучения, если имеются два класса (крестик, нолик) и в нутри крестиков есть один нолик, тогда решающее дерево постарается выделить участок для нолика среди всех крестиков. Естественно, для тестовой выборки в этой выделенной области для нолика - должны лежать крестики, а модель скажет что там нолики, что снизит качество.

Если у дерева много узлов, много листьев, тогда скорее всего оно будет переобученным. Мы хотим чтобы алгоритм построения дерева находил дерево с минимальным числом вершин и наименьшей ошибкой - эта задача является NP-полная.

### Жадное построение дерва

Опишем базовый жадный алгоритм построения бинарного решающего дерева. Запишем алгоримт в виде псевдокода:

Имеется функция $\text{BulidNode}(m, R_m):$, которая на вход принимает номер вершины $m$ и объекты которые попали в данную вершину $R_m$. Функция будет запускаться с аргументами $\text{BulidNode}(1, X)$, где $X$ - это вся обучающая выборка.

**Шаг 1.** Проверяем, необходимо ли продолжать построение дерева. Есил выполнен критерий остановки: 

* тогда считаем прогноз $c_m$ и завершаем работу объявив вершину листовой.

**Шаг 2.** Иначе, необходимо продолжить разбиение. Тогда необходимо найти оптимальный предикат, в качестве предиката мы будем использовать $[x_j < t]$, значит необходимо найти оптимальные $j, t$ - запишем это как:

$$\large j, t = \underset{\substack{j = 1, \ldots, d \\t}}{\text{arg max}} \quad Q(R_m, j, t)$$ - Выполняем $\text{arg max}$ по всем признакам $j$ и по всем порогам $t$, считаем значение функционала качества предиката. Другими словами мы выполняем полный перебор всех признаков, всех порогов и берем ту пару, на которой значение функционала качества для предиката максимальное.
    
Значит, дальше необходимо выполнить разбиение: Множество объектов, которые пойдут влево $R_{l} = \{(x, y) \in R_{m} | [x_{j} \le t]\}$, а вправо $R_{r} = \{(x, y) \in R_{m} | [x_{j} > t]\}$

**Шаг 3.** Запускаем функцию  для левого поддерева $\text{BulidNode}(l, R_{l})$ и правого поддерева $\text{BulidNode}(r, R_{r})$
    

**Критерии остановки:**

* Для задачи классификации  - если среди всех объектов, которые попали в вершину $R_m$, содержатся объекты только одного класса, тогда можно остановиться.

* Если достигнута максимальная глубина

* Если мощность $|R_m| < K$ меньше $K$, то есть, если в $R_m$ содержится мало объектов, тогда нет смысла продолжать разбиение. 


**Как посчитать прогноз $c_m$**

Если мы вершину $c_m$ объявляем листовой, дальше не разбиваем, так как это лист, тогда необходимо выдавать некоторый прогноз на листе:

* Для задачи классификации - Выдаем самый популярный класс в $R_m$

* Для задачи классификации - Выдаем распределение классов в $R_m$

* Для задачи регрессии - Выдаем среднее значение по $R_m$

**Пороги**

Берем $j$-й признак и сортируем все объекты по значению данного признака $x_{1 j} \le \ldots \le x_{\ell j}$. Тогда берем все промежуточные пороги (между каждой парой). Но не понятно какое значение порога брать, можем вспомнить идею о SVM, что нет необходимости делать лишних предположения о данных, разделяя выборки на пополам. Значит мы возьмем все попроги между значениями признаков $t_{1} = \frac{x_{1 j} + x_{2 j}}{2}, \ldots$. 

Из-за большого количества объектов, количество порогов будет огромным и перебор для разбиеня усложниться, так как мы находимо оптимальную пару из $j, t$. Соответственно, для отсортированных значений признаков $x_{1 j} \le \ldots \le x_{\ell j}$ и использовать в качестве порога перцентили, например 100 штук.

<h1 style="color:#008B8B">3. Критерии информативности</h1>

Введем функцию $H(R)$, которая считается конкретно для вершины дерева - это критерий информативности / мера хаотичности вершины (impurity criterion). Это функция показыват насколько неоднородно распределение целевой переменной в этой верщине.

Усрединм по всем объектам, которые попали в множество $R$ и посчитаем ошибку при прогнозировании некоторой константой $c$ и возьмем минимум по всем таким констатам $c$:

$$\large H(R) = \min_{c \in \mathbb{Y}} \frac{1}{|R|} \sum_{(x_i, y_i) \in R} L(y_i, c)$$

Представляем, что эта вершина $R$ будет являться листой и дальше мы ее не будем разбивать, тогда мы должны на этой вершине выдавать константный прогноз. Тогда мы перебираем все возможные константные прогнозы в этой вершине и для каждого констангного прогноза посмотрим насколько он ошибается, какова ошибка константного прогноза на выборке $R$ и выберем такой константный прогноз при котором ошибка будет минимальна и это будет значением impurity.

Если мы решаем задачу классификации и в нашей вершине три положительных объекта, тогда если взять константу $c = +1$, тогда значение функции потерь будет нулевое и impurity тоже будет равен $H(R) = 0$. А если вершина будет разнородная тогда для любой константы значение $H(R)$ окажется больше.

<h2 style="color:#008B8B">3.1 Регрессия</h2>

Как обычно, в регрессии выберем квадрат отклонения в качестве функции потерь $L(y, c) = (y - c)^2$. В этом случае критерий информативности будет выглядеть как:

$$\large H(R) = \min_{c \in \mathbb{Y}} \frac{1}{|R|} \sum_{(x_i, y_i) \in R} (y_i - c)^2$$

Как известно, минимум в этом выражении будет достигаться на среднем значении целевой переменной. Мы значем, что в такой функции оптимальны прогноз $c$ это среднее значение $y$ по всей выборке $R$, запишем как $c = \frac{1}{|R|} \sum\limits_{(x_i, y_i) \in R} y_i = \bar{y}$, тогда $H(R)$ считается как:

$$\large H(R) = \frac{1}{|R|} \sum_{(x_i, y_i) \in R} (y_i - \bar{y})^2$$

Мы получили, что информативность вершины измеряется её дисперсией — чем ниже разброс целевой переменной, тем лучше вершина. Разумеется, можно использовать и другие функции ошибки L — например, при выборе абсолютного отклонения мы получим в качестве критерия среднее абсолютное отклонение от медианы.

Если в вершине находится три объекта с одним значением целевой переменной, тогда дисперсия в этой выборке будет равна 0.

<h2 style="color:#008B8B">3.2 Классификация</h2>